In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sqlalchemy import create_engine

In [2]:
# Set working directory
%cd /Users/liqiuxuan/Desktop/Checkpoint\ 2\ -\ Dataset

/Users/liqiuxuan/Desktop/Checkpoint 2 - Dataset


## Create databases and tables

In [3]:
# Pass the connection url to a variable
conn_url = 'postgresql://liqiuxuan:@localhost/superstore'

# Create the engine and make connectioon to it
engine = create_engine(conn_url)
conn = engine.connect()

In [4]:
# Pass the statements of creating all tables to a variable
create_tables = """
    CREATE TABLE managers (
        manager_id integer,
        manager    varchar(50) NOT NULL,
        PRIMARY KEY (manager_id)
    );    
    
    CREATE TABLE regions (
        region_id  integer,
        region     varchar(15) NOT NULL,
        manager_id integer NOT NULL,
        PRIMARY KEY (region_id),
        FOREIGN KEY (manager_id) REFERENCES managers(manager_id)
    );
    
    CREATE TABLE addresses (
        zipcode    integer,
        city       varchar(40),
        state      varchar(40) NOT NULL,
        region_id  integer NOT NULL,
        PRIMARY KEY (zipcode, city),
        FOREIGN KEY (region_id) REFERENCES regions(region_id)
    );

    CREATE TABLE customers (
        customer_id   integer,
        customer_name varchar(100) NOT NULL,
        zipcode       integer NOT NULL,
        city          varchar(40) NOT NULL,
        PRIMARY KEY (customer_id),
        FOREIGN KEY (zipcode, city) REFERENCES addresses(zipcode, city)
    );

    CREATE TABLE segments (
        segment_id       integer,
        customer_segment varchar(30) NOT NULL,
        PRIMARY KEY (segment_id)
    );

    CREATE TABLE customer_segment (
        customer_id integer,
        segment_id  integer,
        PRIMARY KEY (customer_id, segment_id),
        FOREIGN KEY (customer_id) REFERENCES customers(customer_id),
        FOREIGN KEY (segment_id) REFERENCES segments(segment_id)
    );
    
    CREATE TABLE product_categories (
        category_id     integer,
        category        varchar(60) NOT NULL,
        sub_category    varchar(100) NOT NULL,
        PRIMARY KEY (category_id)
    );
    
    CREATE TABLE products (
        product_id          integer,
        product_name        varchar(200) NOT NULL,
        category_id         integer NOT NULL,
        product_container   varchar(40) NOT NULL,
        unit_price          numeric(8,2) NOT NULL,
        product_base_margin numeric(5,2),
        PRIMARY KEY (product_id),
        FOREIGN KEY (category_id) REFERENCES product_categories(category_id)
    );
    
    CREATE TABLE transactions (
        transaction_id   integer,
        order_id         integer NOT NULL,
        product_id       integer NOT NULL,
        customer_id      integer NOT NULL,
        order_date       date NOT NULL,
        order_priority   varchar(30) NOT NULL,
        discount         numeric(5,2) NOT NULL,
        quantity_ordered integer,
        sales            numeric(10,2) NOT NULL,
        profit           numeric(16,8) NOT NULL,
        PRIMARY KEY (transaction_id),
        FOREIGN KEY (product_id) REFERENCES products(product_id),
        FOREIGN KEY (customer_id) REFERENCES customers(customer_id)
    );
    
    CREATE TABLE shippings (
        shipping_id    integer,
        transaction_id integer NOT NULL,
        ship_mode      varchar(40) NOT NULL,
        ship_cost      numeric(8,2) NOT NULL,
        ship_date      date NOT NULL,
        PRIMARY KEY (shipping_id),
        FOREIGN KEY (transaction_id) REFERENCES transactions(transaction_id)
    );

    CREATE TABLE returns (
        return_id integer,
        order_id integer NOT NULL,
        return_status varchar(20) NOT NULL,
        PRIMARY KEY (return_id)
    );
"""

# Execute the statements for creating tables
conn.execute(create_tables)

## Read data files

Since there are three sheets in the original Excel xlsx file, which are "Orders", "Returns", "Users", we have to convert the sheets in the xlsx file into **".csv"** file form first.

In [5]:
# Read data from the "Orders" sheet
def xlsx_to_csv_pd():
    orders_xls = pd.read_excel('Superstore.xlsx', index_col=0, sheet_name='Orders')
    orders_xls.to_csv('superstore_main.csv', encoding='utf-8')

if __name__ == '__main__':
    xlsx_to_csv_pd()

df_orders = pd.read_csv('superstore_main.csv')

# Read data from the "Returns" sheet
def xlsx_to_csv_pd2():
    returns_xls = pd.read_excel('Superstore.xlsx', index_col=0, sheet_name='Returns')
    returns_xls.to_csv('superstore_returns.csv', encoding='utf-8')

if __name__ == '__main__':
    xlsx_to_csv_pd2()

df_returns = pd.read_csv('superstore_returns.csv')

# Read data from the "Users" sheet
def xlsx_to_csv_pd3():
    users_xls = pd.read_excel('Superstore.xlsx', index_col=0, sheet_name='Users')
    users_xls.to_csv('superstore_users.csv', encoding='utf-8')

if __name__ == '__main__':
    xlsx_to_csv_pd3()

df_users = pd.read_csv('superstore_users.csv')

In [6]:
df_orders.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9426 entries, 0 to 9425
Data columns (total 24 columns):
Row ID                  9426 non-null int64
Order Priority          9426 non-null object
Discount                9426 non-null float64
Unit Price              9426 non-null float64
Shipping Cost           9426 non-null float64
Customer ID             9426 non-null int64
Customer Name           9426 non-null object
Ship Mode               9426 non-null object
Customer Segment        9426 non-null object
Product Category        9426 non-null object
Product Sub-Category    9426 non-null object
Product Container       9426 non-null object
Product Name            9426 non-null object
Product Base Margin     9354 non-null float64
Region                  9426 non-null object
State or Province       9426 non-null object
City                    9426 non-null object
Postal Code             9426 non-null int64
Order Date              9426 non-null object
Ship Date               9426 non-null

In [7]:
df_orders.head()

,Row ID,Order Priority,Discount,Unit Price,Shipping Cost,Customer ID,Customer Name,Ship Mode,Customer Segment,Product Category,...,Region,State or Province,City,Postal Code,Order Date,Ship Date,Profit,Quantity ordered new,Sales,Order ID
0,18606,Not Specified,0.01,2.88,0.50,2,Janice Fletcher,Regular Air,Corporate,Office Supplies,...,Central,Illinois,Addison,60101,2012-05-28,2012-05-30,1.3200,2,5.90,88525
1,20847,High,0.01,2.84,0.93,3,Bonnie Potter,Express Air,Corporate,Office Supplies,...,West,Washington,Anacortes,98221,2010-07-07,2010-07-08,4.5600,4,13.01,88522
2,23086,Not Specified,0.03,6.68,6.15,3,Bonnie Potter,Express Air,Corporate,Office Supplies,...,West,Washington,Anacortes,98221,2011-07-27,2011-07-28,-47.6400,7,49.92,88523
3,23087,Not Specified,0.01,5.68,3.60,3,Bonnie Potter,Regular Air,Corporate,Office Supplies,...,West,Washington,Anacortes,98221,2011-07-27,2011-07-28,-30.5100,7,41.64,88523
4,23088,Not Specified,0.00,205.99,2.50,3,Bonnie Potter,Express Air,Corporate,Technology,...,West,Washington,Anacortes,98221,2011-07-27,2011-07-27,998.2023,8,1446.67,88523


In [8]:
df_users.head()

,Region,Manager
0,Central,Chris
1,East,Erin
2,South,Sam
3,West,William


In [9]:
df_returns.head()

,Order ID,Status
0,65,Returned
1,612,Returned
2,614,Returned
3,678,Returned
4,710,Returned


## Populate database tables

### 1. managers

In [10]:
# Insert column manager_id to df_users
df_users.insert(0, 'manager_id', range(1, len(df_users)+1))
df_users

,manager_id,Region,Manager
0,1,Central,Chris
1,2,East,Erin
2,3,South,Sam
3,4,West,William


In [11]:
# Create the dataframe for managers table and push the data to database
pd.DataFrame({
    'manager_id': df_users['manager_id'],
    'manager': df_users['Manager']
}).drop_duplicates().to_sql(name='managers', con=conn, if_exists='append', index=False)

### 2. regions

In [12]:
# Insert column region_id to df_users
df_users.insert(0, 'region_id', range(1, len(df_users)+1))
df_users

,region_id,manager_id,Region,Manager
0,1,1,Central,Chris
1,2,2,East,Erin
2,3,3,South,Sam
3,4,4,West,William


In [13]:
# Create the dataframe for regions table
regions_df = pd.DataFrame({
    'region_id': df_users['region_id'],
    'region': df_users['Region'],
    'manager_id': df_users['manager_id']
}).drop_duplicates()

In [14]:
# Push the regions data to database
regions_df.to_sql(name='regions', con=conn, if_exists='append', index=False)

### 3. addresses

In [15]:
# Create the region_id_list and add it to df_orders as column region_id
region_id_list = [regions_df['region_id'][regions_df['region'] == i].values[0] for i in df_orders['Region']]

In [16]:
# Add region_id column to df_orders
df_orders.insert(1, 'region_id', region_id_list)
df_orders.head()

,Row ID,region_id,Order Priority,Discount,Unit Price,Shipping Cost,Customer ID,Customer Name,Ship Mode,Customer Segment,...,Region,State or Province,City,Postal Code,Order Date,Ship Date,Profit,Quantity ordered new,Sales,Order ID
0,18606,1,Not Specified,0.01,2.88,0.50,2,Janice Fletcher,Regular Air,Corporate,...,Central,Illinois,Addison,60101,2012-05-28,2012-05-30,1.3200,2,5.90,88525
1,20847,4,High,0.01,2.84,0.93,3,Bonnie Potter,Express Air,Corporate,...,West,Washington,Anacortes,98221,2010-07-07,2010-07-08,4.5600,4,13.01,88522
2,23086,4,Not Specified,0.03,6.68,6.15,3,Bonnie Potter,Express Air,Corporate,...,West,Washington,Anacortes,98221,2011-07-27,2011-07-28,-47.6400,7,49.92,88523
3,23087,4,Not Specified,0.01,5.68,3.60,3,Bonnie Potter,Regular Air,Corporate,...,West,Washington,Anacortes,98221,2011-07-27,2011-07-28,-30.5100,7,41.64,88523
4,23088,4,Not Specified,0.00,205.99,2.50,3,Bonnie Potter,Express Air,Corporate,...,West,Washington,Anacortes,98221,2011-07-27,2011-07-27,998.2023,8,1446.67,88523


In [17]:
# Create the dataframe for addresses table
addresses_df = pd.DataFrame({
    'zipcode': df_orders['Postal Code'],
    'city': df_orders['City'],
    'state': df_orders['State or Province'],
    'region_id': df_orders['region_id']
}).drop_duplicates()

In [18]:
# Push the addresses data to database
addresses_df.to_sql(name='addresses', con=conn, if_exists='append', index=False)

### 4. customers

In [19]:
# Create the dataframe for customers table
customers_df = pd.DataFrame({
    'customer_id': df_orders['Customer ID'],
    'customer_name': df_orders['Customer Name'],
    'zipcode': df_orders['Postal Code'],
    'city': df_orders['City']
}).drop_duplicates()

In [20]:
# Push the customers data to database
customers_df.to_sql(name='customers', con=conn, if_exists='append', index=False)

### 5. segments

In [21]:
# Create the dataframe for segments table
segments_df = pd.DataFrame({'customer_segment': df_orders['Customer Segment']}).drop_duplicates()

In [22]:
# Insert segment_id column
segments_df.insert(0, 'segment_id', range(1, len(segments_df)+1))
segments_df

,segment_id,customer_segment
0,1,Corporate
7,2,Home Office
19,3,Small Business
105,4,Consumer


In [23]:
# Push the segments data to database
segments_df.to_sql(name='segments', con=conn, if_exists='append', index=False)

### 6. customer_segment

In [24]:
# Create segment_id list
segment_id_list = [segments_df['segment_id'][segments_df['customer_segment'] == i].values[0] for i in df_orders['Customer Segment']]

In [25]:
# Add segment_id column to main dataframe df_orders
df_orders.insert(1, 'segment_id', segment_id_list)
df_orders.head()

,Row ID,segment_id,region_id,Order Priority,Discount,Unit Price,Shipping Cost,Customer ID,Customer Name,Ship Mode,...,Region,State or Province,City,Postal Code,Order Date,Ship Date,Profit,Quantity ordered new,Sales,Order ID
0,18606,1,1,Not Specified,0.01,2.88,0.50,2,Janice Fletcher,Regular Air,...,Central,Illinois,Addison,60101,2012-05-28,2012-05-30,1.3200,2,5.90,88525
1,20847,1,4,High,0.01,2.84,0.93,3,Bonnie Potter,Express Air,...,West,Washington,Anacortes,98221,2010-07-07,2010-07-08,4.5600,4,13.01,88522
2,23086,1,4,Not Specified,0.03,6.68,6.15,3,Bonnie Potter,Express Air,...,West,Washington,Anacortes,98221,2011-07-27,2011-07-28,-47.6400,7,49.92,88523
3,23087,1,4,Not Specified,0.01,5.68,3.60,3,Bonnie Potter,Regular Air,...,West,Washington,Anacortes,98221,2011-07-27,2011-07-28,-30.5100,7,41.64,88523
4,23088,1,4,Not Specified,0.00,205.99,2.50,3,Bonnie Potter,Express Air,...,West,Washington,Anacortes,98221,2011-07-27,2011-07-27,998.2023,8,1446.67,88523


In [26]:
# Create the dataframe for customer_segment table
customer_segment_df = pd.DataFrame({
    'customer_id': df_orders['Customer ID'],
    'segment_id': df_orders['segment_id']
}).drop_duplicates()

In [27]:
# Push the customer_segment data to database
customer_segment_df.to_sql(name='customer_segment', con=conn, if_exists='append', index=False)

### 7. product_categories

In [28]:
# Create the dataframe for product_categories table
product_categories_df = pd.DataFrame({
    'category': df_orders['Product Category'],
    'sub_category': df_orders['Product Sub-Category']
}).drop_duplicates()

In [29]:
# Insert category_id column to the table
product_categories_df.insert(0, 'category_id', range(1, len(product_categories_df)+1))
product_categories_df.head()

,category_id,category,sub_category
0,1,Office Supplies,Labels
1,2,Office Supplies,Pens & Art Supplies
2,3,Office Supplies,Paper
3,4,Office Supplies,"Scissors, Rulers and Trimmers"
4,5,Technology,Telephones and Communication


In [31]:
# Push the product_categories data to database
product_categories_df.to_sql(name='product_categories', con=conn, if_exists='append', index=False)

### 8. products

In [32]:
# Create the category_id_list
category_id_list = [product_categories_df['category_id'][product_categories_df['sub_category'] == i].values[0] for i in df_orders['Product Sub-Category']]

In [33]:
# Add category_id column to df_orders
df_orders.insert(1, 'category_id', category_id_list)
df_orders.head()

,Row ID,category_id,segment_id,region_id,Order Priority,Discount,Unit Price,Shipping Cost,Customer ID,Customer Name,...,Region,State or Province,City,Postal Code,Order Date,Ship Date,Profit,Quantity ordered new,Sales,Order ID
0,18606,1,1,1,Not Specified,0.01,2.88,0.50,2,Janice Fletcher,...,Central,Illinois,Addison,60101,2012-05-28,2012-05-30,1.3200,2,5.90,88525
1,20847,2,1,4,High,0.01,2.84,0.93,3,Bonnie Potter,...,West,Washington,Anacortes,98221,2010-07-07,2010-07-08,4.5600,4,13.01,88522
2,23086,3,1,4,Not Specified,0.03,6.68,6.15,3,Bonnie Potter,...,West,Washington,Anacortes,98221,2011-07-27,2011-07-28,-47.6400,7,49.92,88523
3,23087,4,1,4,Not Specified,0.01,5.68,3.60,3,Bonnie Potter,...,West,Washington,Anacortes,98221,2011-07-27,2011-07-28,-30.5100,7,41.64,88523
4,23088,5,1,4,Not Specified,0.00,205.99,2.50,3,Bonnie Potter,...,West,Washington,Anacortes,98221,2011-07-27,2011-07-27,998.2023,8,1446.67,88523


In [34]:
# Create the dataframe for products table
products_df = pd.DataFrame({
    'product_name': df_orders['Product Name'],
    'category_id': df_orders['category_id'],
    'product_container': df_orders['Product Container'],
    'unit_price': df_orders['Unit Price'],
    'product_base_margin': df_orders['Product Base Margin']
}).drop_duplicates()

In [35]:
# Add product_id column to the table
products_df.insert(0, 'product_id', range(1, len(products_df)+1))
products_df.head()

,product_id,product_name,category_id,product_container,unit_price,product_base_margin
0,1,Avery 49,1,Small Box,2.88,0.36
1,2,SANFORD Liquid Accent™ Tank-Style Highlighters,2,Wrap Bag,2.84,0.54
2,3,Xerox 1968,3,Small Box,6.68,0.37
3,4,Acme® Preferred Stainless Steel Scissors,4,Small Pack,5.68,0.56
4,5,V70,5,Small Box,205.99,0.59


In [36]:
# Push the products data to database
products_df.to_sql(name='products', con=conn, if_exists='append', index=False)

### 9. transactions

In [37]:
# Create the product_id list
product_id_list = [products_df['product_id'][products_df['product_name'] == i].values[0] for i in df_orders['Product Name']]

In [38]:
# Add product_id column to df_orders
df_orders.insert(1, 'product_id', product_id_list)
df_orders.head()

,Row ID,product_id,category_id,segment_id,region_id,Order Priority,Discount,Unit Price,Shipping Cost,Customer ID,...,Region,State or Province,City,Postal Code,Order Date,Ship Date,Profit,Quantity ordered new,Sales,Order ID
0,18606,1,1,1,1,Not Specified,0.01,2.88,0.50,2,...,Central,Illinois,Addison,60101,2012-05-28,2012-05-30,1.3200,2,5.90,88525
1,20847,2,2,1,4,High,0.01,2.84,0.93,3,...,West,Washington,Anacortes,98221,2010-07-07,2010-07-08,4.5600,4,13.01,88522
2,23086,3,3,1,4,Not Specified,0.03,6.68,6.15,3,...,West,Washington,Anacortes,98221,2011-07-27,2011-07-28,-47.6400,7,49.92,88523
3,23087,4,4,1,4,Not Specified,0.01,5.68,3.60,3,...,West,Washington,Anacortes,98221,2011-07-27,2011-07-28,-30.5100,7,41.64,88523
4,23088,5,5,1,4,Not Specified,0.00,205.99,2.50,3,...,West,Washington,Anacortes,98221,2011-07-27,2011-07-27,998.2023,8,1446.67,88523


In [39]:
# Create the dataframe for transactions table
transactions_df = pd.DataFrame({
    'transaction_id': df_orders['Row ID'],
    'order_id': df_orders['Order ID'],
    'product_id': df_orders['product_id'],
    'customer_id': df_orders['Customer ID'],
    'order_date': df_orders['Order Date'],
    'order_priority': df_orders['Order Priority'],
    'discount': df_orders['Discount'],
    'quantity_ordered': df_orders['Quantity ordered new'],
    'sales': df_orders['Sales'],
    'profit': df_orders['Profit']
}).drop_duplicates()

In [41]:
# Push the transactions data to database
transactions_df.to_sql(name='transactions', con=conn, if_exists='append', index=False)

### 10. shippings

In [42]:
# Create dataframe for shippings table and add shipping_id column
shippings_df = pd.DataFrame({
    'transaction_id': df_orders['Row ID'],
    'ship_mode': df_orders['Ship Mode'],
    'ship_cost': df_orders['Shipping Cost'],
    'ship_date': df_orders['Ship Date']
}).drop_duplicates()
shippings_df.insert(0, 'shipping_id', range(1, len(shippings_df)+1))
shippings_df.head()

,shipping_id,transaction_id,ship_mode,ship_cost,ship_date
0,1,18606,Regular Air,0.50,2012-05-30
1,2,20847,Express Air,0.93,2010-07-08
2,3,23086,Express Air,6.15,2011-07-28
3,4,23087,Regular Air,3.60,2011-07-28
4,5,23088,Express Air,2.50,2011-07-27


In [43]:
# Push the shippings data to database
shippings_df.to_sql(name='shippings', con=conn, if_exists='append', index=False)

### 11. returns

In [44]:
# Create the dataframe for returns table and add the return_id column
returns_df = pd.DataFrame({
    'order_id': df_returns['Order ID'],
    'return_status': df_returns['Status']
}).drop_duplicates()
returns_df.insert(0, 'return_id', range(1, len(returns_df)+1))
returns_df.head()

,return_id,order_id,return_status
0,1,65,Returned
1,2,612,Returned
2,3,614,Returned
3,4,678,Returned
4,5,710,Returned


In [45]:
# Push the returns data to database
returns_df.to_sql(name='returns', con=conn, if_exists='append', index=False)